# Machine Learning with Shogun

#### By Saurabh Mahindre - <a href="https://github.com/Saurabh7">github.com/Saurabh7</a> as a part of <a href="http://www.google-melange.com/gsoc/project/details/google/gsoc2014/saurabh7/5750085036015616">Google Summer of Code 2014 project</a> mentored by - Heiko Strathmann - <a href="https://github.com/karlnapf">github.com/karlnapf</a> - <a href="http://herrstrathmann.de/">herrstrathmann.de</a>

In this notebook we will see how machine learning problems are generally represented and solved in Shogun. As a primer to Shogun's many capabilities, we will see how various types of data and its attributes are handled and also how prediction is done. 

1. [Introduction](#Introduction)
2. [Using datasets](#Using-datasets)
3. [Feature representations](#Feature-representations)
4. [Labels](#Assigning-labels)
5. [Preprocessing data](#Preprocessing-data)
6. [Supervised Learning with Shogun's Machine interface](#supervised)
7. [Evaluating performance and Model selection](#Evaluating-performance-and-Model-selection)
8. [Example: Regression](#More-predictions:-Regression)

### Introduction

Machine learning concerns the construction and study of systems that can learn from data via exploiting certain types of structure within these. The uncovered patterns are then used to predict future data, or to perform other kinds of decision making. Two main classes (among others) of Machine Learning algorithms are: predictive or [supervised](http://en.wikipedia.org/wiki/Supervised_learning) learning and descriptive or [Unsupervised](http://en.wikipedia.org/wiki/Unsupervised_learning) learning. Shogun provides functionality to address those (and more) problem classes.

In [ ]:
import os
SHOGUN_DATA_DIR=os.getenv('SHOGUN_DATA_DIR', '../../../data')
import shogun as sg
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In a general problem setting for the supervised learning approach, the goal is to learn a mapping from inputs $x_i\in\mathcal{X} $ to outputs $y_i \in \mathcal{Y}$, given a labeled set of input-output pairs $ \mathcal{D} = {(x_i,y_i)}^{\text N}_{i=1} $$\subseteq \mathcal{X} \times \mathcal{Y}$. Here $ \mathcal{D}$ is called the training set, and $\text N$ is the number of training examples. In the simplest setting, each training input $x_i$ is a  $\mathcal{D}$ -dimensional vector of numbers, representing, say, the height and weight of a person. These are called $\textbf {features}$, attributes or covariates. In general, however, $x_i$ could be a complex structured object, such as an image.<ul><li>When the response variable $y_i$ is categorical and discrete, $y_i \in$ {1,...,C} (say male or female) it is a [classification](http://en.wikipedia.org/wiki/Classification_in_machine_learning) problem.</li><li>When it is continuous (say the prices of houses) it is a [regression](http://en.wikipedia.org/wiki/Regression_analysis) problem.</li></ul>
For the unsupervised learning
approach we are only given inputs,  $\mathcal{D} = {(x_i)}^{\text N}_{i=1}$ , and the goal is to find “interesting
patterns” in the data. 

### Using datasets

Let us consider an example, we have a dataset about various attributes of individuals and we know whether or not they are diabetic. The data reveals certain configurations of attributes that correspond to diabetic patients and others that correspond to non-diabetic patients. When given a set of attributes for a new patient, the goal is to predict whether the patient is diabetic or not. This type of learning problem falls under [Supervised learning](http://en.wikipedia.org/wiki/Supervised_learning), in particular, [classification](http://en.wikipedia.org/wiki/Classification_in_machine_learning).

Shogun provides the capability to load datasets of different formats using [File](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1File.html).</br>  A real world dataset: [Pima Indians Diabetes data set](http://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes) is used now. We load the `LibSVM` format file using Shogun's [LibSVMFile](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CLibSVMFile.html) class. The `LibSVM` format is: $$\space \text {label}\space \text{attribute1:value1 attribute2:value2 }...$$$$\space.$$$$\space .$$ LibSVM uses the so called "sparse" format where zero values do not need to be stored.

In [ ]:
#Load the file
data_file=sg.read_libsvm(os.path.join(SHOGUN_DATA_DIR, 'uci/diabetes/diabetes_scale.svm'))

This results in a [LibSVMFile](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CLibSVMFile.html) object which we will later use to access the data.

### Feature representations

To get off the mark, let us see how Shogun handles the attributes of the data using [Features](http://shogun-toolbox.org/api/latest/classshogun_1_1Features.html) class. Shogun supports wide range of feature representations. We believe it is a good idea to have different forms of data, rather than converting them all into matrices. Among these are: $\hspace {20mm}$<ul><li>[String features](http://www.shogun-toolbox.org/doc/en/current/classshogun_1_1CStringFeatures.html): Implements a list of strings. Not limited to character strings, but could also be sequences of floating point numbers etc. Have varying dimensions. </li> <li>[Dense features](http://www.shogun-toolbox.org/doc/en/current/classshogun_1_1DenseFeatures.html): Implements dense feature matrices</li> <li>[Sparse features](http://www.shogun-toolbox.org/doc/en/current/classshogun_1_1SparseFeatures.html):  Implements sparse matrices.</li><li>[Streaming features](http://shogun-toolbox.org/doc/en/latest/classshogun_1_1StreamingFeatures.html): For algorithms working on data streams (which are too large to fit into memory) </li></ul> 

`SpareRealFeatures` (sparse features handling `64 bit float` type data) are used to get the data from the file. Since `LibSVM` format files have labels included in the file, `load_with_labels` method of `SpareRealFeatures` is used. In this case it is interesting to play with two attributes, Plasma glucose concentration and Body Mass Index (BMI) and try to learn something about their relationship with the disease. We get hold of the feature matrix using `get_full_feature_matrix` and row vectors 1 and 5 are extracted. These are the attributes we are interested in.

In [ ]:
f=sg.SparseRealFeatures()
trainlab=f.load_with_labels(data_file)
mat=f.get_full_feature_matrix()

#exatract 2 attributes
glucose_conc=mat[1]
BMI=mat[5]

#generate a numpy array
feats=np.vstack((glucose_conc, BMI))
print(feats, feats.shape)

In numpy, this is a matrix of 2 row-vectors of dimension 768. However, in Shogun, this will be a matrix of 768 column vectors of dimension 2. This is beacuse each data sample is stored in a column-major fashion, meaning each column here corresponds to an individual sample and each row in it to an atribute like BMI, Glucose concentration etc. To convert the extracted matrix into Shogun format, `RealFeatures` are used which are nothing but the above mentioned [Dense features](http://www.shogun-toolbox.org/doc/en/current/classshogun_1_1DenseFeatures.html) of `64bit Float` type. To do this call the factory method, `features` with the  matrix (this should be a 64bit 2D numpy array) as the argument. 

In [ ]:
#convert to shogun format
feats_train = sg.create_features(feats)

Some of the general methods you might find useful are:

* `get("feature_matrix")`: The feature matrix can be accessed using this.
* `get("num_features")`: The total number of attributes can be accesed using this.
* `get_num_vectors()`: To get total number of samples in data.
* `feat_matrix[:,idx]`: To get all the attribute values (A.K.A [feature vector](http://en.wikipedia.org/wiki/Feature_vector)) for a particular sample by passing the index of the sample as argument.</li></ul>

In [ ]:
#Get number of features(attributes of data) and num of vectors(samples)
feat_matrix=feats_train.get("feature_matrix")
num_f=feats_train.get("num_features")
num_s=feats_train.get_num_vectors()

print('Number of attributes: %s and number of samples: %s' %(num_f, num_s))
print('Number of rows of feature matrix: %s and number of columns: %s' %(feat_matrix.shape[0], feat_matrix.shape[1]))
print('First column of feature matrix (Data for first individual):')
print(feat_matrix[:,0])

### Assigning labels

In supervised learning problems, training data is labelled. Shogun provides various types of labels to do this through [Clabels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1Labels.html). Some of these are:<ul><li>[Binary labels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1BinaryLabels.html): Binary Labels for binary classification which can have values +1 or -1.</li><li>[Multiclass labels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1MulticlassLabels.html): Multiclass Labels for multi-class classification which can have values from 0 to (num. of classes-1).</li><li>[Regression labels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1RegressionLabels.html):  Real-valued labels used for regression problems and are returned as output of classifiers.</li><li>[Structured labels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1StructuredLabels.html): Class of the labels used in Structured Output (SO) problems</li></ul></br> In this particular problem, our data can be of two types: diabetic or non-diabetic, so we need [binary labels](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1BinaryLabels.html). This makes it a [Binary Classification problem](http://en.wikipedia.org/wiki/Binary_classification), where the data has to be classified in two groups.

In [ ]:
#convert to shogun format labels
labels=sg.create_labels(trainlab)

The labels can be accessed using `get_labels` and the confidence vector using `get_values`. The total number of labels is available using `get_num_labels`.

In [ ]:
n=labels.get_num_labels()
print('Number of labels:', n)

### Preprocessing data

It is usually better to preprocess data to a standard form rather than handling it in raw form. The reasons are having a well behaved-scaling, many algorithms assume centered data, and that sometimes one wants to de-noise data (with say PCA). Preprocessors do not change the domain of the input features. It is possible to do various type of preprocessing using methods provided by [Preprocessor](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1Preprocessor.html) class. Some of these are:<ul><li>[Norm one](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CNormOne.html): Normalize vector to have norm 1.</li><li>[PruneVarSubMean](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CPruneVarSubMean.html):  Substract the mean and remove features that have zero variance. </li><li>[Dimension Reduction](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CDimensionReductionPreprocessor.html): Lower the dimensionality of given simple features.<ul><li>[PCA](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CPCA.html): Principal component analysis.</li><li>[Kernel PCA](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1KernelPCA.html): PCA using kernel methods.</li></ul></li></ul> The training data will now be preprocessed using [CPruneVarSubMean](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CPruneVarSubMean.html). This will basically remove data with zero variance and subtract the mean. Passing a `True` to the constructor makes the class normalise the varaince of the variables. It basically dividies every dimension through its standard-deviation. This is the reason behind removing dimensions with constant values.  It is required to initialize the preprocessor by passing the feature object to `init` before doing anything else. The raw and processed data is now plotted.

In [ ]:
preproc=sg.create_transformer("PruneVarSubMean", divide_by_std=True)
preproc.fit(feats_train)
feats_train = preproc.transform(feats_train)
# Store preprocessed feature matrix.
preproc_data=feats_train.get("feature_matrix")

In [ ]:
# Plot the raw training data.
plt.figure(figsize=(13,6))
pl1=plt.subplot(121)
plt.gray()
_=plt.scatter(feats[0, :], feats[1,:], c=labels.get("labels"), s=50,
              cmap=matplotlib.colors.ListedColormap(["blue", "red"]))
plt.vlines(0, -1, 1, linestyle='solid', linewidths=2)
plt.hlines(0, -1, 1, linestyle='solid', linewidths=2)
plt.title("Raw Training Data")
_=plt.xlabel('Plasma glucose concentration')
_=plt.ylabel('Body mass index')
p1 = plt.Rectangle((0, 0), 1, 1, fc="r")
p2 = plt.Rectangle((0, 0), 1, 1, fc="b")
pl1.legend((p1, p2), ["Non-diabetic", "Diabetic"], loc=2)

#Plot preprocessed data.
pl2=plt.subplot(122)
_=plt.scatter(preproc_data[0, :], preproc_data[1,:], c=labels.get("labels"), s=50,
              cmap=matplotlib.colors.ListedColormap(["blue", "red"]))
plt.vlines(0, -5, 5, linestyle='solid', linewidths=2)
plt.hlines(0, -5, 5, linestyle='solid', linewidths=2)
plt.title("Training data after preprocessing")
_=plt.xlabel('Plasma glucose concentration')
_=plt.ylabel('Body mass index')
p1 = plt.Rectangle((0, 0), 1, 1, fc="r")
p2 = plt.Rectangle((0, 0), 1, 1, fc="b")
pl2.legend((p1, p2), ["Non-diabetic", "Diabetic"], loc=2)

Horizontal and vertical lines passing through zero are included to make the processing of data clear. Note that the now processed data has zero mean.

### <a id='supervised'>Supervised Learning with Shogun's <a href='http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1Machine.html'>Machine</a> interface</a>

[Machine](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1Machine.html) is Shogun's interface for general learning machines. Basically one has to ` train()`  the machine on some training data to be able to learn from it. Then we `apply()` it to test data to get predictions. Some of these are: <ul><li>[Kernel machine](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1KernelMachine.html): Kernel based learning tools.</li><li>[Linear machine](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1LinearMachine.html): Interface for all kinds of linear machines like classifiers.</li><li>[Distance machine](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1DistanceMachine.html): A distance machine is based on a a-priori choosen distance.</li><li>[Gaussian process machine](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1GaussianProcessMachine.html): A base class for Gaussian Processes. </li><li>And many more</li></ul>

Moving on to the prediction part, [Liblinear](http://www.shogun-toolbox.org/doc/en/current/classshogun_1_1LibLinear.html), a linear SVM is used to do the classification (more on SVMs in [this notebook](http://www.shogun-toolbox.org/static/notebook/current/SupportVectorMachines.html)). A linear SVM will find a linear separation with the largest possible margin. Here C is a penalty parameter on the loss function. 

In [ ]:
#prameters to svm
C=0.9

svm=sg.create_machine("LibLinear", C1=C, C2=C,
               liblinear_solver_type="L2R_L2LOSS_SVC")
#train
svm.train(feats_train, labels)

size=100

We will now `apply` on test features to get predictions. For visualising the classification boundary, the whole XY is used as test data, i.e. we predict the class on every point in the grid.

In [ ]:
x1=np.linspace(-5.0, 5.0, size)
x2=np.linspace(-5.0, 5.0, size)
x, y=np.meshgrid(x1, x2)
#Generate X-Y grid test data
grid=sg.create_features(np.array((np.ravel(x), np.ravel(y))))

#apply on test grid
predictions = svm.apply(grid)
#get output labels
z=predictions.get_values().reshape((size, size))

#plot
plt.jet()
plt.figure(figsize=(9,6))
plt.title("Classification")
c=plt.pcolor(x, y, z)
_=plt.contour(x, y, z, linewidths=1, colors='black', hold=True)
_=plt.colorbar(c)

_=plt.scatter(preproc_data[0, :], preproc_data[1,:], c=trainlab, s=50,
              cmap=matplotlib.colors.ListedColormap(["blue", "red"]))
_=plt.xlabel('Plasma glucose concentration')
_=plt.ylabel('Body mass index')
plt.p1 = plt.Rectangle((0, 0), 1, 1, fc="r")
plt.p2 = plt.Rectangle((0, 0), 1, 1, fc="b")
plt.legend((p1, p2), ["Non-diabetic", "Diabetic"], loc=2)
plt.show()

Let us have a look at the weight vector of the separating hyperplane. It should tell us about the linear relationship between the features. The decision boundary is now plotted by solving for $\bf{w}\cdot\bf{x}$ + $\text{b}=0$. Here $\text b$ is a bias term which allows the linear function to be offset from the origin of the used coordinate system. Methods `get_w()` and `get_bias()` are used to get the necessary values.

In [ ]:
w=svm.get("w")
b=svm.get("bias")

x1=np.linspace(-2.0, 3.0, 100)

#solve for w.x+b=0
def solve (x1):
    return -( ( (w[0])*x1 + b )/w[1] )
x2=list(map(solve, x1))

#plot
plt.figure(figsize=(7,6))
plt.plot(x1,x2, linewidth=2)
plt.title("Decision boundary using w and bias")
_=plt.scatter(preproc_data[0, :], preproc_data[1,:], c=trainlab, s=50,
              cmap=matplotlib.colors.ListedColormap(["blue", "red"]))
_=plt.xlabel('Plasma glucose concentration')
_=plt.ylabel('Body mass index')
p1 = plt.Rectangle((0, 0), 1, 1, fc="r")
p2 = plt.Rectangle((0, 0), 1, 1, fc="b")
plt.legend((p1, p2), ["Non-diabetic", "Diabetic"], loc=2)

print('w :', w)
print('b :', b)

For this problem, a linear classifier does a reasonable job in distinguishing labelled data. An interpretation could be that individuals below a certain level of BMI and glucose are likely to have no Diabetes. 
For problems where the data cannot be separated linearly, there are more advanced classification methods, as for example all of Shogun's [kernel machines](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1KernelMachine.html), but more on this later. To play with this interactively have a look at this: [web demo](http://demos.shogun-toolbox.org/classifier/binary/) 

### Evaluating performance and Model selection

How do you assess the quality of a prediction? Shogun provides various ways to do this using [CEvaluation](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CEvaluation.html). The preformance is evaluated by comparing the predicted output and the expected output. Some of the base classes for performance measures are:

* [Binary class evaluation](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CBinaryClassEvaluation.html): used to evaluate binary classification labels. 
* [Clustering evaluation](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1ClusteringEvaluation.html): used to evaluate clustering.
* [Mean absolute error](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CMeanAbsoluteError.html): used to compute an error of regression model.
* [Multiclass accuracy](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1MulticlassAccuracy.html): used to compute accuracy of multiclass classification. 

Evaluating on training data should be avoided since the learner may adjust to very specific random features of the training data which are not very important to the general relation. This is called [overfitting](http://en.wikipedia.org/wiki/Overfitting). Maximising performance on the training examples usually results in algorithms explaining the noise in data (rather than actual patterns), which leads to bad performance on unseen data. The dataset will now be split into two, we train on one part and evaluate performance on other using [CAccuracyMeasure](http://shogun-toolbox.org/api/latest/classshogun_1_1CAccuracyMeasure.html).

In [ ]:
#split features for training and evaluation
num_train=700
feats=np.array(glucose_conc)
feats_t=feats[:num_train]
feats_e=feats[num_train:]
feats=np.array(BMI)
feats_t1=feats[:num_train]
feats_e1=feats[num_train:]
feats_t=np.vstack((feats_t, feats_t1))
feats_e=np.vstack((feats_e, feats_e1))

feats_train = sg.create_features(feats_t)
feats_evaluate = sg.create_features(feats_e)

Let's see the accuracy by applying on test features.

In [ ]:
label_t=trainlab[:num_train]
labels=sg.create_labels(label_t)
label_e=trainlab[num_train:]
labels_true=sg.create_labels(label_e)

svm=sg.create_machine("LibLinear", C1=C, C2=C,
               liblinear_solver_type="L2R_L2LOSS_SVC")

#train and evaluate
svm.train(feats_train, labels)
output=svm.apply(feats_evaluate)

#use AccuracyMeasure to get accuracy
acc=sg.create_evaluation("AccuracyMeasure")
accuracy=acc.evaluate(output,labels_true)*100
print('Accuracy(%):', accuracy)

To evaluate more efficiently [cross-validation](http://en.wikipedia.org/wiki/Cross-validation_%28statistics%29) is used. As you might have wondered how are the parameters of the classifier selected? Shogun has a model selection framework to select the best parameters. More description of these things in [this notebook](http://shogun-toolbox.org/notebook/latest/xval_modelselection.html).

### More predictions: Regression

This section will demonstrate another type of machine learning problem on real world data.</br> The task is to estimate prices of houses in Boston using the [Boston Housing Dataset](https://archive.ics.uci.edu/ml/datasets/Housing) provided by [StatLib library](http://lib.stat.cmu.edu/DASL/). The attributes are: Weighted distances to employment centres and percentage lower status of the population. Let us see if we can predict a good relationship between the pricing of houses and the attributes. This type of problems are solved using [Regression analysis](http://en.wikipedia.org/wiki/Regression_analysis).

The data set is now loaded using [LibSVMFile](http://www.shogun-toolbox.org/doc/en/latest/classshogun_1_1CLibSVMFile.html) as in the previous sections and the attributes required (7th and 12th vector ) are converted to Shogun format features.

In [ ]:
temp_feats = sg.create_features(sg.read_csv(os.path.join(SHOGUN_DATA_DIR, 'uci/housing/fm_housing.dat')))
labels=sg.create_labels(sg.read_csv(os.path.join(SHOGUN_DATA_DIR, 'uci/housing/housing_label.dat')))

#rescale to 0...1
preproc=sg.create_transformer("RescaleFeatures")
preproc.fit(temp_feats)
temp_feats = preproc.transform(temp_feats)
mat = temp_feats.get("feature_matrix")

dist_centres=mat[7]
lower_pop=mat[12]

feats=np.array(dist_centres)
feats=np.vstack((feats, np.array(lower_pop)))
print(feats, feats.shape)
#convert to shogun format features
feats_train = sg.create_features(feats)

The tool we will use here to perform regression is [Kernel ridge regression](http://shogun-toolbox.org/doc/en/latest/classshogun_1_1KernelRidgeRegression.html). Kernel Ridge Regression is a non-parametric version of ridge regression where the [kernel trick](http://en.wikipedia.org/wiki/Kernel_trick) is used to solve a related linear ridge regression problem in a higher-dimensional space, whose results correspond to non-linear regression in the data-space.  Again we train on the data and apply on the XY grid to get predicitions.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
size=100
x1=np.linspace(0, 1.0, size)
x2=np.linspace(0, 1.0, size)
x, y=np.meshgrid(x1, x2)
#Generate X-Y grid test data
grid = sg.create_features(np.array((np.ravel(x), np.ravel(y))))

#Train on data(both attributes) and predict
width=1.0
tau=0.5
kernel=sg.create_kernel("GaussianKernel", width=width)
krr=sg.create_machine("KernelRidgeRegression",tau=tau, kernel=kernel, labels=labels)
krr.train(feats_train)
kernel.init(feats_train, grid)
out = krr.apply().get("labels")


The `out` variable now contains a relationship between the attributes. Below is an attempt to establish such relationship between the attributes individually. Separate feature instances are created for each attribute. You could skip the code and have a look at the plots directly if you just want the essence.  

In [ ]:
#create feature objects for individual attributes.
feats_test = sg.create_features(x1.reshape(1,len(x1)))
feats_t0=np.array(dist_centres)
feats_train0 = sg.create_features(feats_t0.reshape(1,len(feats_t0)))
feats_t1=np.array(lower_pop)
feats_train1 = sg.create_features(feats_t1.reshape(1,len(feats_t1)))

#Regression with first attribute
kernel=sg.create_kernel("GaussianKernel", width=width)
krr=sg.create_machine("KernelRidgeRegression",tau=tau, kernel=kernel, labels=labels)
krr.train(feats_train0)
kernel.init(feats_train0, feats_test)
out0 = krr.apply().get("labels")

#Regression with second attribute 
kernel=sg.create_kernel("GaussianKernel", width=width)
krr=sg.create_machine("KernelRidgeRegression",tau=tau, kernel=kernel, labels=labels)
krr.train(feats_train1)
kernel.init(feats_train1, feats_test)
out1 = krr.apply().get("labels")

In [ ]:
#Visualization of regression
fig=plt.figure(figsize=(20,6))
#first plot with only one attribute
fig.add_subplot(131)
plt.title("Regression with 1st attribute")
_=plt.scatter(feats[0, :], labels.get("labels"), cmap="gray", s=20)
_=plt.xlabel('Weighted distances to employment centres ')
_=plt.ylabel('Median value of homes')

_=plt.plot(x1,out0, linewidth=3)

#second plot with only one attribute
fig.add_subplot(132)
plt.title("Regression with 2nd attribute")
_=plt.scatter(feats[1, :], labels.get("labels"), cmap="gray", s=20)
_=plt.xlabel('% lower status of the population')
_=plt.ylabel('Median value of homes')
_=plt.plot(x1,out1, linewidth=3)

#Both attributes and regression output
ax=fig.add_subplot(133, projection='3d')
z=out.reshape((size, size))
plt.gray()
plt.title("Regression")
ax.plot_wireframe(y, x, z, linewidths=2, alpha=0.4)
ax.set_xlabel('% lower status of the population')
ax.set_ylabel('Distances to employment centres ')
ax.set_zlabel('Median value of homes')
ax.view_init(25, 40)